# Decision Tree

## Implementation

### Import libraries 
Let's first import the libraries, mainly `pandas`, `numpy` and Decision Tree implementation from `sklearn`:

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

### Load results datasets

Now, load the original dataset, taking the targets apart from the features:

In [2]:
# Train dataset
dataset_train = pd.read_csv('datasets/covertype_norm_train.csv')
# Targets
target_train = dataset_train.iloc[:,-1]
# Dataset without classes
data_train = dataset_train.iloc[:,:-1]
data_train.head()

,elevation,aspect,slope,horiz_dist_hydro,vert_dist_hydro,horiz_dist_road,hillshade_9,hill_shade_noon,hill_shade_15,horiz_dist_fire,...,soil_type_30,soil_type_31,soil_type_32,soil_type_33,soil_type_34,soil_type_35,soil_type_36,soil_type_37,soil_type_38,soil_type_39
0,-0.573753,-0.518424,-0.428658,0.436024,-0.475092,-0.979056,0.927864,0.144520,-0.534162,-0.220768,...,-0.14199,-0.214265,-0.202489,-0.039088,-0.081433,-0.016657,-0.044107,-0.220216,-0.219696,-0.172986
1,1.656009,-0.010549,0.868502,-0.516497,-0.280544,1.817610,0.862413,0.665801,-0.534162,2.273548,...,-0.14199,-0.214265,4.938531,-0.039088,-0.081433,-0.016657,-0.044107,-0.220216,-0.219696,-0.172986
2,0.169501,-0.799569,0.632655,0.455170,1.891910,-0.388051,0.796962,-1.245563,-1.335438,-0.687429,...,-0.14199,-0.214265,-0.202489,-0.039088,-0.081433,-0.016657,-0.044107,-0.220216,-0.219696,-0.172986
3,-1.205043,1.268208,1.576043,0.234990,1.648725,-0.649457,-2.933743,-0.159560,1.956291,-0.501856,...,-0.14199,-0.214265,-0.202489,-0.039088,-0.081433,-0.016657,-0.044107,-0.220216,-0.219696,-0.172986
4,-1.057345,0.152697,0.986425,0.134472,0.530073,-1.041945,0.404256,1.056762,-0.014415,-0.794770,...,-0.14199,-0.214265,-0.202489,-0.039088,-0.081433,-0.016657,-0.044107,-0.220216,-0.219696,-0.172986


In [3]:
# Test dataset
dataset_test = pd.read_csv('datasets/covertype_norm_test.csv')
# Targets
target_test = dataset_test.iloc[:,-1]
# Dataset without classes
data_test = dataset_test.iloc[:,:-1]
data_test.head()

,elevation,aspect,slope,horiz_dist_hydro,vert_dist_hydro,horiz_dist_road,hillshade_9,hill_shade_noon,hill_shade_15,horiz_dist_fire,...,soil_type_30,soil_type_31,soil_type_32,soil_type_33,soil_type_34,soil_type_35,soil_type_36,soil_type_37,soil_type_38,soil_type_39
0,-1.350358,1.730737,-0.782429,-0.889847,-0.783127,-0.407751,-0.381155,0.231400,0.570300,-0.351760,...,-0.14199,-0.214265,-0.202489,-0.039088,-0.081433,-0.016657,-0.044107,-0.220216,-0.219696,-0.172986
1,-0.857238,0.787540,1.104349,-0.253237,1.081293,-0.297127,-1.722900,1.360843,1.848011,-0.840253,...,-0.14199,-0.214265,-0.202489,-0.039088,-0.081433,-0.016657,-0.044107,-0.220216,-0.219696,-0.172986
2,0.422017,1.794222,0.632655,0.459957,0.303100,1.027330,-1.101116,-1.028362,0.332083,0.262267,...,-0.14199,-0.214265,-0.202489,-0.039088,-0.081433,-0.016657,-0.044107,-0.220216,-0.219696,-0.172986
3,-1.698163,1.277277,3.109050,-0.277170,1.373115,-1.150296,-4.733644,-1.549644,2.151197,-1.108606,...,-0.14199,-0.214265,-0.202489,-0.039088,-0.081433,-0.016657,-0.044107,-0.220216,-0.219696,-0.172986
4,-0.630926,1.812360,-0.782429,1.063061,0.416587,-0.907074,-0.282979,0.101080,0.440364,-0.429082,...,-0.14199,-0.214265,-0.202489,-0.039088,-0.081433,-0.016657,-0.044107,-0.220216,-0.219696,-0.172986


Now, load the dataset with the results from the **genetic** and **pso** algorithm executions:

In [4]:
# Read selected attributes of PSO
ga  = pd.read_csv('results/ga_selected_attributes.csv')
# Read selected attributes of pso
pso  = pd.read_csv('results/pso_selected_attributes.csv')
# Read selected attributes of filtered
filtered  = pd.read_csv('results/filter_selected_attrs.csv')

### Decision Tree definition

In [5]:
def perform_decision_tree_selector(attr):
    '''
    Performs decision tree for a given dataset.
    '''
    columns = attr.columns.tolist()[:-54] + ['accuracy']
    new_df = pd.DataFrame(columns=columns)
    
    for index in range(len(attr)):
        vector = attr.iloc[index,-54:].tolist()
        sliced_train = data_train.iloc[:, vector]       
        sliced_test = data_test.iloc[:, vector]
        
        # Perform decision tree
        d_tree = DecisionTreeClassifier(random_state=0)
        d_tree.fit(sliced_train, target_train)
        new_df.loc[index] = attr.iloc[0,:-54].tolist() + [d_tree.score(sliced_test, target_test)]
    
    return new_df

def perform_decision_tree_extractor(train, test):
    '''
    Performs decision tree for a given dataset.
    '''
    
    train_target = train['cover_type']
    train_data   = train.loc[:, train.columns != 'cover_type']
    test_target  = test['cover_type']
    test_data    = test.loc[:, test.columns != 'cover_type']
    
    d_tree = DecisionTreeClassifier(random_state=0)
    d_tree.fit(train_data, train_target)
    
    return d_tree.score(test_data, test_target)

In [18]:
# Perform decision tree for PSO
dtree_pso_result = perform_decision_tree_selector(pso)

In [19]:
full_dataset = pd.DataFrame(columns=(['n_attr'] + data_train.columns.tolist()))
full_dataset.loc[0] = [54] + [True for i in range(54)]

# Perform decision tree for ORIGINAL DATA
dtree_original_result = perform_decision_tree_selector(full_dataset)

In [20]:
# Perform decision tree for GA
dtree_ga_result = perform_decision_tree_selector(ga)


In [21]:
# Perform decision tree for filtered
dtree_filtered_result = perform_decision_tree_selector(filtered)
dtree_filtered_result

,accuracy
0,0.730033
1,0.774750
2,0.783486


In [23]:
# Perform decision tree for PCA
representativity = [75,85,95]
dtree_pca_result = pd.DataFrame(columns=['representativity (%)', 'accuracy'])
index = 0
for r in representativity:
    train = pd.read_csv('results/pca_' + str(r) + '.csv')
    test  = pd.read_csv('results/pca_test_' + str(r) + '.csv')
    dtree_pca_result.loc[index] = [0r, perform_decision_tree_extractor(train, test)]
    index += 1
    
# Store results of PCA
dtree_pca_result.to_csv('results/decision_tree_pca.csv')
dtree_pca_result

,representativity (%),accuracy
0,75.0,0.759567
1,85.0,0.771423
2,95.0,0.765183


In [24]:
# Perform decision tree for LDA
n_components = range(1,7)
dtree_lda_result = pd.DataFrame(columns=(['n_components', 'accuracy']))
index = 0
for n in n_components:
    train = pd.read_csv('results/lda_' + str(n) + '.csv')
    test  = pd.read_csv('results/lda_test_' + str(n) + '.csv')
    dtree_lda_result.loc[index] = [int(n), perform_decision_tree_extractor(train, test)]
    index += 1
    
# Store results of LDA
dtree_lda_result.to_csv('results/decision_tree_lda.csv')
dtree_lda_result

,n_components,accuracy
0,1.0,0.491057
1,2.0,0.617720
2,3.0,0.693220
3,4.0,0.717554
4,5.0,0.743968
5,6.0,0.760191


In [22]:
print("original: ", dtree_original_result['accuracy'].max())
print("pso: ", dtree_pso_result['accuracy'].max())
print("ga: ", dtree_ga_result['accuracy'].max())
print("filtered: ", dtree_filtered_result['accuracy'].max())
print("pca: ", dtree_pca_result['accuracy'].max())
print("lda: ", dtree_lda_result['accuracy'].max())

original:  0.812603993344426
pso:  0.8053244592346089
ga:  0.752287853577371
filtered:  0.783485856905158
pca:  0.7714226289517471
lda:  0.7601913477537438


In [17]:
# Store results
dtree_pso_result.to_csv('results/decision_tree_pso.csv')
dtree_ga_result.to_csv('results/decision_tree_ga.csv')
dtree_filtered_result.to_csv('results/decision_tree_filtered.csv')
dtree_pca_result.to_csv('results/decision_tree_pca.csv')
dtree_lda_result.to_csv('results/decision_tree_lda.csv')